A simple CNN model. 


In [1]:
# imports
import numpy as np
import pandas
import cv2
import io
import urllib.request
import matplotlib.pyplot as plt
import math
from google.colab import files
from PIL import Image

In [2]:
uploaded = files.upload()

In [3]:
imgSize = 144

In [4]:
# load data
# from skimage.feature import hog
# from skimage import color

data = pandas.read_csv(io.BytesIO(uploaded['SF_SOLD_AUG_2016.csv']))
tempdata = data.iloc[:1000]


labels = ['PROPTYPE', 'STYLE'] # outputs
features = ['PHOTOURL'] # the input

labels = []
features =  []

for url in tempdata.PHOTOURL.values:
  response = urllib.request.urlopen(url)
  image = Image.open(io.BytesIO(response.read()))
  imgArray = np.asarray(image, dtype=np.uint8)
  imgArray = cv2.resize(imgArray, (imgSize, imgSize))
  #gray = color.rgb2gray(imgArray)
  #fd = hog(image=gray)
  features.append(imgArray)

for proptype, style in zip(tempdata.PROPTYPE.values, tempdata.STYLE.values):
  temp = []
  #temp.append(proptype)
  if style == style:
    if ',' in style:
      temp.append(style[:style.index(',')])
    else:
      temp.append(style)
  else:
    temp.append("")
  labels.append(temp)

features = np.array(features)
labels = np.array(labels)

KeyError: ignored

In [ ]:
print(labels.shape)
print(features.shape)

In [ ]:
# split data sets
# from sklearn.preprocessing import MultiLabelBinarizer
# from sklearn.model_selection import train_test_split

mlb = MultiLabelBinarizer()
mlb.fit(labels)

transformedLabels = mlb.transform(labels)

featuresTrain, featuresTest, labelTrain, labelTest = train_test_split(features, transformedLabels, train_size=0.8)

transformedLabels.shape

In [ ]:
# model imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# model let's say a 5 layer cnn

featuresTrain = featuresTrain/255.0

model = Sequential()

model.add(Conv2D(128, (3, 3), input_shape=features.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))

model.add(Dense(transformedLabels.shape[1])) # transformedLabels.shape[1]
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(featuresTrain, labelTrain, epochs=15, validation_split=0.5)

In [ ]:
predicted = model.predict(featuresTest/255.0)

mse = mean_squared_error(labelTest, predicted)
print('mse: ', mse)

r2 = r2_score(labelTest, predicted)
print('r2: ', r2)